In [1]:
# importacao de bibliotecas
import pandas as pd
from unidecode import unidecode
import numpy as np
from sqlalchemy import create_engine

%load_ext sql

In [2]:
# Buscar dados dos arquivos do excel

df_excel_total = pd.read_excel(r'G:\OneDrive - Default Directory\OneDrive - rede.sp\Arquivos_CCIH\ARQUIVOS_ORIGINAIS\Relatorio_CCIH_15_08_ate_03_10_2024.xlsx')

#df_excel_total = pd.read_excel(r'C:\\Users\H013026\Downloads\erro.xlsx')

In [3]:


columns_to_drop = ['Unnamed: 0']

df_excel_total = df_excel_total.drop(columns_to_drop,errors='ignore', axis=1)
df_excel_total.shape[0]

556

In [4]:
#Apresentacao dos dados

df_excel_total.head()

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado
0,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-08-26,1UT6,UTI_6º AND,10854608,ADALBERTO CORREA,1948-06-14,UROCULTURA,Isolado [1]: Klebsiella pneumoniae
1,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-06,2ONC,ONCOLOGIA_4º AND - AMB,10865876,ADEMIR MARTINS DOS SANTOS,1945-12-07,UROCULTURA,Isolado [1]: Escherichia coli
2,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-14,1UT6,UTI_6º AND,10856280,ADEMIR VALERIO,1947-11-22,UROCULTURA,Isolado [1]: Candida albicans
3,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-21,1PCH,CHOQUE,10837256,ADERLANDO RIBEIRO DOS SANTOS,1941-10-16,HEMOCULTURA 1º AMOSTRA,Isolado [1]: Staphylococcus epidermidis
4,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-21,1PCH,CHOQUE,10837257,ADERLANDO RIBEIRO DOS SANTOS,1941-10-16,HEMOCULTURA 2º AMOSTRA,Isolado [1]: Staphylococcus epidermidis


In [5]:
# Busca dos resultados de exames positivados de MDR de com duas linhas no dataframe original
df_excel = df_excel_total
df_resultados_linha_2 = df_excel.loc[((df_excel['Resultado'] == 'Enterococcus resistente a glicopeptideos (VRE), consultar CCIH.') | (df_excel['Resultado'] == 'Cepa resistente a carbapenemicos, consultar CCIH.')| (df_excel['Resultado'] == 'Enterococcus Vanco Resistente (VRE), consultar CCIH.')| (df_excel['Resultado'] == 'Ampicilina - Resistente')|(df_excel['Resultado'] == 'Enterococcus resistente a Vancomicina')) & (df_excel['NomePaciente'] == df_excel['NomePaciente'].shift(1))]

df_resultados_linha_1 = df_excel.loc[(df_resultados_linha_2.index - 1)]

# Alteração do índice no dataframe que buscou os dados do excel para substituição da linha 2 para a linhas 1
df_resultados_linha_2.index = df_resultados_linha_2.index - 1

df_resultados_linha_1['ComplementoResultado'] = df_resultados_linha_2.Resultado.values

df_resultados_linha_1.shape[0]

32

In [6]:
# Busca dos resultados de exames positivados de MDR de com apenas 1 linha no dataframe original
df_excel_2 = df_excel_total

df_resultados_apenas_linha_1 = df_excel_2.loc[((df_excel_2['Resultado'] == 'Enterococcus resistente a glicopeptideos (VRE), consultar CCIH.') | (df_excel_2['Resultado'] == 'Cepa resistente a carbapenemicos, consultar CCIH.')| (df_excel_2['Resultado'] == 'Enterococcus Vanco Resistente (VRE), consultar CCIH.')| (df_excel_2['Resultado'] == 'Ampicilina - Resistente')|(df_excel_2['Resultado'] == 'Enterococcus resistente a Vancomicina')) & (df_excel_2['NomePaciente'] != df_excel_2['NomePaciente'].shift(1))].copy()

df_resultados_apenas_linha_1['ComplementoResultado'] = df_resultados_apenas_linha_1['Resultado']

df_resultados_apenas_linha_1.loc[:, 'Resultado'] = np.nan

df_resultados_apenas_linha_1.shape[0]

0

In [7]:
# Dataframe com todos os resultados de MDR positivados

df_resultados_excel = pd.concat([df_resultados_linha_1, df_resultados_apenas_linha_1], ignore_index=True)

In [8]:
## Conexão com a base de dados postgree

import pyodbc as podbc

cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins18;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")

In [9]:
## Acesso a base de dados

sql = "SELECT * FROM [Isolamento_Versao_2].[dbo].[CCIH] WHERE    [DataSistema] BETWEEN '2024-15-08' AND '2024-03-10' "
df_resultados_sql = pd.read_sql(sql,cnxn)
df_resultados_sql.shape[0]

<ipython-input-9-f105247399b7>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados_sql = pd.read_sql(sql,cnxn)


54

In [10]:
# from excel
# df_resultados_excel

# from data base 
# df_resultados_sql

df_resultados_inner_join = pd.merge(df_resultados_excel, df_resultados_sql, on=['NumeroPedido', 'Resultado'], how='inner')
df_resultados_inner_join.shape[0]


41

In [11]:
# Perform a left join
df_merge_sql = pd.merge(df_resultados_sql, df_resultados_excel, on=['NumeroPedido', 'Resultado'], how='left', indicator=True)
df_merge_sql.shape[0]

54

In [12]:
# Finding columns that contain '__y'
columns_to_drop_sql = [col for col in df_merge_sql.columns if '_y' in col]
# Adding another column to drop
additional_columns_sql = ['_merge'] 
# Append or extend the list with new column na
columns_to_drop_sql.extend(additional_columns_sql)


df_anti_join_sql= df_merge_sql[df_merge_sql['_merge'] == 'left_only'].drop(columns=columns_to_drop_sql)
df_anti_join_sql.shape[0]

13

In [13]:
# Perform a left join
df_merge_excel = pd.merge(df_resultados_excel, df_resultados_sql, on=['NumeroPedido', 'Resultado'], how='left', indicator=True)
# Finding columns that contain '__y'
columns_to_drop_excel = [col for col in df_merge_sql.columns if '_y' in col]

# List of additional columns to drop
additional_columns_excel = ['_merge','posicao','Prontuario','id'] 
# Append or extend the list with new column names
columns_to_drop_excel.extend(additional_columns_excel) 
df_anti_join_excel = df_merge_excel[df_merge_excel['_merge'] == 'left_only'].drop(columns=columns_to_drop_excel)
df_anti_join_excel.shape[0]

3

In [14]:
cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins18;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")
cursor = cnxn.cursor()
try:
    for index, row in df_resultados_inner_join.iterrows():
        # Prepare the SQL query
        sql = """
        UPDATE [Isolamento_Versao_2].[dbo].[CCIH]
        SET [ComplementoResultado] = ?
        WHERE [NumeroPedido] = ? and [Resultado] = ?;
        """
        # Execute the query with parameters
        cursor.execute(sql, (row['ComplementoResultado_x'], row['NumeroPedido'] ,row['Resultado']))
         
    # Commit all updates
    cnxn.commit()
    print("All updates were successful.")
except Exception as e:
    print("Error occurred:", e)
    cnxn.rollback()  # Roll back in case of any error
finally:
    cursor.close()
    cnxn.close()

All updates were successful.


In [15]:
df_anti_join_excel.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
df_anti_join_excel

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado,ComplementoResultado,Ativo
11,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-27,11A,11º AND_- ENF,10860378,GENY IPPOLITO COLPAERT,1946-08-24,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
33,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-26,10A,10º AND_ UTI,10863835,MARIA CARDOSO DE OLIVEIRA,1947-06-20,UROCULTURA,Isolado [2]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
39,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-27,1PS,PRONTO SOCORRO ADULTO_1º AND,10871937,SONIA MARIA LINO DE ABREU,1955-03-02,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN


In [16]:
cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins18;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")
cursor = cnxn.cursor()
try:
    for index, row in df_anti_join_excel.iterrows():
        if pd.isna(row['Resultado']):
      # Prepare the SQL query
            sql = """
            INSERT INTO [dbo].[CCIH]
               (
               [parametroDataInicial]
               ,[parametroDataFinal]
               ,[ParametroUsuarioLogado]
               ,[DataSistema]
               ,[Clinica]
               ,[NomeClinica]
               ,[NumeroPedido]
               ,[NomePaciente]
               ,[DataNascimento]
               ,[Nome]
               , [Resultado]
           
          
               ,[ComplementoResultado]
               ,[Ativo])
         VALUES
               (?,?,?,?,?,?,?,?,?,?,NULL,?,'A')
            """
            # Execute the query with parameters
            cursor.execute(sql, (row['parametroDataInicial'], row['parametroDataFinal'] ,row['ParametroUsuarioLogado'],row['DataSistema'], row['Clinica'] ,row['NomeClinica'],row['NumeroPedido'], row['NomePaciente'] ,row['DataNascimento'],row['Nome'],row['ComplementoResultado'] ))
         
        else:
            # Prepare the SQL query
            sql = """
            INSERT INTO [dbo].[CCIH]
               (
               [parametroDataInicial]
               ,[parametroDataFinal]
               ,[ParametroUsuarioLogado]
               ,[DataSistema]
               ,[Clinica]
               ,[NomeClinica]
               ,[NumeroPedido]
               ,[NomePaciente]
               ,[DataNascimento]
               ,[Nome]
               ,[Resultado]
           
          
               ,[ComplementoResultado]
               ,[Ativo])
         VALUES
               (?,?,?,?,?,?,?,?,?,?,?,?,'A')
            """
            # Execute the query with parameters
            cursor.execute(sql, (row['parametroDataInicial'], row['parametroDataFinal'] ,row['ParametroUsuarioLogado'],row['DataSistema'], row['Clinica'] ,row['NomeClinica'],row['NumeroPedido'], row['NomePaciente'] ,row['DataNascimento'],row['Nome'], row['Resultado'] ,row['ComplementoResultado'] ))
         
    # Commit all updates
    cnxn.commit()
    print("All updates were successful.")
except Exception.Error as e:
    print("Error occurred:", e)
    cnxn.rollback()  # Roll back in case of any error
finally:
    cursor.close()
    cnxn.close()

All updates were successful.


In [17]:

df_anti_join_excel

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado,ComplementoResultado,Ativo
11,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-27,11A,11º AND_- ENF,10860378,GENY IPPOLITO COLPAERT,1946-08-24,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
33,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-26,10A,10º AND_ UTI,10863835,MARIA CARDOSO DE OLIVEIRA,1947-06-20,UROCULTURA,Isolado [2]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
39,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-27,1PS,PRONTO SOCORRO ADULTO_1º AND,10871937,SONIA MARIA LINO DE ABREU,1955-03-02,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN


In [18]:
df_anti_join_sql

,posicao,parametroDataInicial_x,parametroDataFinal_x,ParametroUsuarioLogado_x,DataSistema_x,Clinica_x,NomeClinica_x,NumeroPedido,NomePaciente_x,DataNascimento_x,Nome_x,Resultado,Prontuario,id,ComplementoResultado_x,Ativo
1,None,2024-08-06,2024-09-12,HENRIQLUSTOSA,2024-09-09,10A,10º AND_ UTI,10858398,RAMON CALLEZ GONZALEZ,1937-11-07,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,10306975,4571,"Cepa resistente a carbapenemicos, consultar CCIH.",A
2,None,2024-08-06,2024-09-12,HENRIQLUSTOSA,2024-09-09,10A,10º AND_ UTI,10858398,RAMON CALLEZ GONZALEZ,1937-11-07,UROCULTURA,Isolado [2]: Pseudomonas aeruginosa,10306975,4572,"Cepa resistente a carbapenemicos, consultar CCIH.",A
5,None,2024-08-10,2024-09-19,HENRIQLUSTOSA,2024-09-16,6URO,UROLOGIA_6º AND - AMB,10869811,BENEDITO ROBERTO LEANDRO,1949-12-16,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,10413680,4580,"Cepa resistente a carbapenemicos, consultar CCIH.",A
7,None,2024-08-10,2024-09-19,HENRIQLUSTOSA,2024-09-15,1UT6,UTI_6º AND,10849481,FRANCISCO ANIBAL SIQUEIRA DE MEDEIROS,1945-06-21,HEMOCULTURA 1º AMOSTRA,Isolado [1]: Acinetobacter baumannii,1120690,4582,"Cepa resistente a carbapenemicos, consultar CCIH.",A
10,None,2024-08-10,2024-09-19,HENRIQLUSTOSA,2024-09-15,1UT7,UTI_7º AND,10861053,JOSE CARLOS PEREIRA,1969-04-20,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,12147922,4585,"Cepa resistente a carbapenemicos, consultar CCIH.",A
19,None,2024-08-10,2024-09-19,HENRIQLUSTOSA,2024-09-13,0,SEM CLINICA CADASTRADA,10853901,RAQUEL ANA DE OLIVEIRA.,1964-03-22,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,4129847,4594,"Cepa resistente a carbapenemicos, consultar CCIH.",A
20,None,2024-08-10,2024-09-19,HENRIQLUSTOSA,2024-09-11,9C,9º AND_C ENFERMARIA GERAL,10854547,SANDRA HELENA DANIEL,1972-09-28,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,9512393,4595,"Cepa resistente a carbapenemicos, consultar CCIH.",A
39,None,2024-07-01,2024-09-02,HENRIQLUSTOSA,2024-08-26,1PS2,PRONTO SOCORRO _2º AND,10860701,LUIZ RICARDO NATAL,1966-01-17,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,12111415,4565,"Cepa resistente a carbapenemicos, consultar CCIH.",A
48,None,2024-08-04,2024-09-06,HENRIQLUSTOSA,2024-08-31,1UT7,UTI_7º AND,10851452,SUZETE POLLETO FRAZAO,1950-09-28,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,1868751,4569,"Cepa resistente a carbapenemicos, consultar CCIH.",A
49,None,2024-08-09,2024-09-18,HENRIQLUSTOSA,2024-09-16,6URO,UROLOGIA_6º AND - AMB,10869811,BENEDITO ROBERTO LEANDRO,1949-12-16,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,10413680,4574,"Cepa resistente a carbapenemicos, consultar CCIH.",A


In [19]:
df_resultados_inner_join

,parametroDataInicial_x,parametroDataFinal_x,ParametroUsuarioLogado_x,DataSistema_x,Clinica_x,NomeClinica_x,NumeroPedido,NomePaciente_x,DataNascimento_x,Nome_x,...,DataSistema_y,Clinica_y,NomeClinica_y,NomePaciente_y,DataNascimento_y,Nome_y,Prontuario,id,ComplementoResultado_y,Ativo
0,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-24,1PS,PRONTO SOCORRO ADULTO_1º AND,10868152,ALMIRA CONCEIÇAO,1959-05-28,UROCULTURA,...,2024-09-24,1PS,PRONTO SOCORRO ADULTO_1º AND,ALMIRA CONCEIÇAO,1959-05-28,UROCULTURA,2142024,4605,Enterococcus resistente a glicopeptideos (VRE)...,A
1,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-18,1UT7,UTI_7º AND,10839252,CHARLES WESLEY ALVES DE OLIVEIRA,1963-11-07,HEMOCULTURA 2º AMOSTRA,...,2024-09-18,1UT7,UTI_7º AND,CHARLES WESLEY ALVES DE OLIVEIRA,1963-11-07,HEMOCULTURA 2º AMOSTRA,4620266,4597,"Cepa resistente a carbapenemicos, consultar CCIH.",A
2,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-17,0,SEM CLINICA CADASTRADA,10866106,CHARLES WESLEY ALVES DE OLIVEIRA,1963-11-07,CULTURA GERAL,...,2024-09-17,0,SEM CLINICA CADASTRADA,CHARLES WESLEY ALVES DE OLIVEIRA,1963-11-07,CULTURA GERAL,4620266,4598,"Cepa resistente a carbapenemicos, consultar CCIH.",A
3,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-05,1UT6,UTI_6º AND,10849507,EDGARD ANTONIO,1965-03-04,HEMOCULTURA SANGUE PERIFERICO,...,2024-09-05,1UT6,UTI_6º AND,EDGARD ANTONIO,1965-03-04,HEMOCULTURA SANGUE PERIFERICO,3466935,4570,"Cepa resistente a carbapenemicos, consultar CCIH.",A
4,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-05,1UT6,UTI_6º AND,10849507,EDGARD ANTONIO,1965-03-04,HEMOCULTURA SANGUE PERIFERICO,...,2024-09-05,1UT6,UTI_6º AND,EDGARD ANTONIO,1965-03-04,HEMOCULTURA SANGUE PERIFERICO,3466935,4581,"Cepa resistente a carbapenemicos, consultar CCIH.",A
5,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-08-16,11A,11º AND_- ENF,10853374,EDNA MARINI DE CAMPOS,1967-05-17,CULTURA GERAL,...,2024-08-16,11A,11º AND_- ENF,EDNA MARINI DE CAMPOS,1967-05-17,CULTURA GERAL,8004806,4579,Enterococcus resistente a Vancomicina,A
6,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-08-15,12A,12º AND_ - ENF,10844092,ERALDO FELICIANO DA SILVA,1948-07-18,CULTURA GERAL,...,2024-08-15,12A,12º AND_ - ENF,ERALDO FELICIANO DA SILVA,1948-07-18,CULTURA GERAL,10511830,4556,"Cepa resistente a carbapenemicos, consultar CCIH.",A
7,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-23,1PS,PRONTO SOCORRO ADULTO_1º AND,10868621,ERIVALDO PEDRO DA SILVA,1952-08-29,UROCULTURA,...,2024-09-23,1PS,PRONTO SOCORRO ADULTO_1º AND,ERIVALDO PEDRO DA SILVA,1952-08-29,UROCULTURA,1256593,4599,"Cepa resistente a carbapenemicos, consultar CCIH.",A
8,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-15,1UT6,UTI_6º AND,10849482,FRANCISCO ANIBAL SIQUEIRA DE MEDEIROS,1945-06-21,HEMOCULTURA 2º AMOSTRA,...,2024-09-15,1UT6,UTI_6º AND,FRANCISCO ANIBAL SIQUEIRA DE MEDEIROS,1945-06-21,HEMOCULTURA 2º AMOSTRA,1120690,4583,"Cepa resistente a carbapenemicos, consultar CCIH.",A
9,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-15,1UT6,UTI_6º AND,10849482,FRANCISCO ANIBAL SIQUEIRA DE MEDEIROS,1945-06-21,HEMOCULTURA 2º AMOSTRA,...,2024-09-15,1UT6,UTI_6º AND,FRANCISCO ANIBAL SIQUEIRA DE MEDEIROS,1945-06-21,HEMOCULTURA 2º AMOSTRA,1120690,4576,"Cepa resistente a carbapenemicos, consultar CCIH.",A


In [20]:
df_anti_join_excel

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado,ComplementoResultado,Ativo
11,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-27,11A,11º AND_- ENF,10860378,GENY IPPOLITO COLPAERT,1946-08-24,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
33,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-26,10A,10º AND_ UTI,10863835,MARIA CARDOSO DE OLIVEIRA,1947-06-20,UROCULTURA,Isolado [2]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
39,2024-08-15,2024-10-03,HENRIQLUSTOSA,2024-09-27,1PS,PRONTO SOCORRO ADULTO_1º AND,10871937,SONIA MARIA LINO DE ABREU,1955-03-02,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN


In [21]:
df_anti_join_excel.size


39